In [1]:
import pymongo
import pandas as pd
import numpy as np
import pyecharts

In [2]:
def parseKuan():
    client = pymongo.MongoClient('localhost',27017)
    db = client['appKuAn']
    collection = db['KuanItem']
    # 将数据库转为dataframe
    data = pd.DataFrame(list(collection.find()))
    print(data.shape)
    print(data.dtypes)
    print(data.info())
    print(data.head())
    return data
originData = parseKuan()
originData.drop('_id',axis = 1, inplace = True)
originData.head()

(5776, 9)
_id          object
comment      object
download     object
follow       object
name         object
num_score    object
score        object
tags         object
volume       object
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5776 entries, 0 to 5775
Data columns (total 9 columns):
_id          5776 non-null object
comment      5776 non-null object
download     5776 non-null object
follow       5776 non-null object
name         5776 non-null object
num_score    5776 non-null object
score        5776 non-null object
tags         5776 non-null object
volume       5776 non-null object
dtypes: object(9)
memory usage: 406.2+ KB
None
                        _id comment download follow          name num_score  \
0  5c324c949b9e83a126a8110a    1982     936万   4220      WiFi万能钥匙      2064   
1  5c324c949b9e83a126a8110c    5308     421万   6355         讯飞输入法      1865   
2  5c324c949b9e83a126a8110e    4653     549万   1.0万       ES文件浏览器      5488   
3  5c324c959b9e83a126

,comment,download,follow,name,num_score,score,tags,volume
0,1982,936万,4220,WiFi万能钥匙,2064,2.3,[wifi],25.52M
1,5308,421万,6355,讯飞输入法,1865,4.3,"[输入法, 手写, 语音, 讯飞, iFlytek, 小编推荐]",19.85M
2,4653,549万,1.0万,ES文件浏览器,5488,4.2,"[文件管理, es管理器, 装机必备]",21.49M
3,5830,429万,3311,搜狗输入法,1918,2.8,"[搜狗输入法, 输入法, 拼音, 手写, sogou]",31.10M
4,6162,321万,4450,联通手机营业厅(官方版),1351,2.5,"[中国联通, 营业厅, 运营商]",36.50M


## 文本型数据的转换

In [3]:
data = originData.copy()
def clean_symbol(x):
    if '万' in str(x):
        return int(float(str(x).replace('万', '')) * 10000)
    else:
        return int(x)

def clean_symbol2(x):
    if 'M' in str(x):
        return float(str(x).replace('M', ''))
    else:
        return float(str(x)[:-1]) 

# data['download'] = data['download'].map(clean_symbol)
# data['follow'] = data['follow'].map(clean_symbol)  
# data['comment'] = data['comment'].map(clean_symbol)
# data['num_score'] = data['num_score'].map(clean_symbol)
data['volume'] = data['volume'].map(clean_symbol2)
# data[['comment','num_score']] = data[['comment','num_score']].applymap(int)
data['score'] = data['score'].map(float)
data[['download', 'follow', 'comment', 'num_score']] = data[['download', 'follow', 'comment', 'num_score']].applymap(clean_symbol)
analyseData = data[['comment', 'download', 'follow', 'num_score', 'score', 'volume']]
analyseData.describe()

,comment,download,follow,num_score,score,volume
count,5776.000000,5.776000e+03,5776.000000,5776.000000,5776.000000,5776.000000
mean,274.242209,1.472368e+05,775.950831,141.797784,3.901316,40.063378
std,1524.666493,1.036726e+06,1981.305672,623.542143,0.619980,115.164936
min,0.000000,2.200000e+01,0.000000,1.000000,1.600000,0.010000
25%,16.000000,1.893750e+03,69.000000,6.000000,3.700000,4.932500
50%,40.000000,8.506500e+03,195.000000,18.000000,4.000000,13.985000
75%,127.000000,5.000000e+04,621.000000,73.000000,4.300000,31.220000
max,54000.000000,5.401000e+07,38000.000000,17000.000000,4.800000,998.920000


# 从总体和分类两个维度对 App 下载量、评分、体积等指标进行分析

## 总体情况

### 下载量排名

In [5]:
# 下载量分布
bins = np.array([0, 10, 100, 500, 10000]) * 10000
groups = ['<=10万', '10-100万', '100-500万', '>500万']
cats = pd.cut(analyseData['download'], bins, labels = groups )
cats = pd.value_counts(cats)
from pyecharts import Bar
bar = Bar(title = 'App 下载数量区间分布', subtitle = '绝大部分 App 下载量低于 10万')
bar.add('App 数量 （个）', list(cats.index), list(cats.values), is_label_show = True, is_splitline_show = False)
# bar.render(path = 'download_interval.png', pixel_ration = 1)
bar

In [33]:
another = analyseData.copy()
another = another.sort_values(by = 'download',ascending= False)
yIndex = data['name'][another[:21].index]
number = another[:21]['download'] / 10000
score = another[:21]['score']
bar2 = pyecharts.Bar('下载量最多的20个App', is_animation = True)
bar2.add('download',yIndex.values ,number ,is_label_show = True,
         is_splitline_show = False,is_convert = True, is_more_utils=True, label_pos = 'right',xaxis_rotate = 45)
bar2